## `Threading in Python`

## ***Author: [Tisha Jhabak](https://github.com/TishaJhabak1014)***

>This notebook will illustrate the intuition of threading with examples as we walk through the notebook.

In [25]:
import time                  # importing the time module

start = time.perf_counter()   # starting a counter to keep track of how long it takes to run the script below

def do_something():
  print('Sleeping for 1 second...')
  time.sleep(1)
  print('Done Sleeping...')

do_something()

finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

Sleeping for 1 second...
Done Sleeping...
Finished in 1.0 second(s)


Running the code, it said `Finished in 1.0 second(s)` which is pretty much right as we are running the `do_something()` function one time. 
Let us see what happens on running the function twice :

In [26]:
start = time.perf_counter()   # starting a counter to keep track of how long it takes to run the script below

do_something()
do_something()

finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

Sleeping for 1 second...


09:45:14: Thread 1: finishing


Done Sleeping...
Sleeping for 1 second...
Done Sleeping...
Finished in 2.0 second(s)


So, we can see that each time we run the `do_something` function in our script, it is adding about 1 second to our script. Here, it is not really doing something in the CPU when the function runs. Our script is just waiting around for a second and when that is done, it sets around to wait another second sleeping. As it is done, the script above finishes. Pictorially, the execution of the above script can be depicted as below:

![](https://raw.githubusercontent.com/CoreyMSchafer/code_snippets/692ccae54a47a16c8286d91b08707150c20531fc/Python/Threading/threading-1.svg)

Running everything in order like this is called `running synchronous`. So, everytime our program runs synchronously, it actually is not doing much on the CPU. It just sits around waiting around like this. Thst is not actually a good way. Here is where we leverage  the use of threading and concurrency. These are called CPU-bound and I/O bound task. CPU task are the one which are using a lot of numbers and CPU. I/O bound task are basically those which just waits for input and output operations to be completed and are not really using the CPU much. Some I/O based tasks are reading and writing from and to file systems, network operations, downloading things from the internet.


Generally,
- If our task are I/O based, threading helps running our script faster.
- However, if our task is doing a lot of data crunching and are CPU-bound, then there is probably not much benefit by the use of threading.
- As a matter of fact, some programs actually runs slower using threads because of the added overhead cost when they are creating and storing different threads.
- So, if our taskis CPU bound we are more likely to use multi-processing and run it in parallel instead.

So, when we run something concurrently using threads, it is not actually going to run the code at the same time. It just gives the illusion of running the code at the same time because when it comes to wait, it is just waiting around, it is just going to go ahead and move forward with the script and run the other code when the I/O operations finishes. The representation can be as:

![](https://raw.githubusercontent.com/CoreyMSchafer/code_snippets/692ccae54a47a16c8286d91b08707150c20531fc/Python/Threading/threading-2.svg)

There is actually a overlap there but it is never actually running the code at the same time. It is just going to give the illusion of it. And finally we can see we are done a little bit sooner.


## Implementation of Threading

We can use the standard python library `threading` for this.

In [27]:
import threading  # importing the thread module

start = time.perf_counter()
t1 = threading.Thread(target=do_something) # passing the functions as argument
t2 = threading.Thread(target=do_something)  # instead of calling the function twice, turning them into threads

finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

Finished in 0.0 second(s)


Running the above script, we see that our script ended immediately, but nothing actually printed out, so our function did not actually ran. 

To get our threads to run, we need to use atrat method on each thread as :

In [28]:
start = time.perf_counter()

t1 = threading.Thread(target=do_something) # passing the functions as argument
t2 = threading.Thread(target=do_something)  # instead of calling the function twice, turning them into threads

t1.start()
t2.start()

finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

Sleeping for 1 second...Sleeping for 1 second...
Finished in 0.01 second(s)



Here, the reason that it is saying that our script got finished in 0 second is because it started both of the threads and while the sleep statements got executed, the script ran concurrently and continue running the rest of the script. Do, it immediately came down and calculated the finish time while the threads where sleeping. Now, once the sleep time was done, the threads finally gor fully executed.

What if we wanted our threads to get finished entirely before moving to a next statement of counting the finish time? In that case we use the method, `join()`.

In [29]:
start = time.perf_counter()

t1 = threading.Thread(target=do_something) # passing the functions as argument
t2 = threading.Thread(target=do_something)  # instead of calling the function twice, turning them into threads

t1.start()
t2.start()

t1.join()
t2.join()

finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

Sleeping for 1 second...
Sleeping for 1 second...
Done Sleeping...
Done Sleeping...
Done Sleeping...
Done Sleeping...
Finished in 1.01 second(s)


Then, we see both the threads started at the almost exact same time and then printed out they were done sleeping. Then, our script printed out the finish time. Also, our code ran in two seconds before, and now it is running in 1 second.

Now what if we want too call the function `do_something` 10 times? Doing it without threads will take almost 10 seconds as it will run synchronously, i.e one after the other, so before the next line of the script starts the previous lines must be completely executed. But if we use threads to run it 10 times, we still get the finish time around 1 second.

In [30]:
start = time.perf_counter()

threads = []

for _ in range(10):                 # note that the _ is to signify that it is a throwaway variable
    do_something()

finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

Sleeping for 1 second...
Done Sleeping...
Sleeping for 1 second...
Done Sleeping...
Sleeping for 1 second...
Done Sleeping...
Sleeping for 1 second...
Done Sleeping...
Sleeping for 1 second...
Done Sleeping...
Sleeping for 1 second...
Done Sleeping...
Sleeping for 1 second...
Done Sleeping...
Sleeping for 1 second...
Done Sleeping...
Sleeping for 1 second...
Done Sleeping...
Sleeping for 1 second...
Done Sleeping...
Finished in 10.02 second(s)


In [31]:
start = time.perf_counter()

threads = []

for _ in range(10):                 # note that the _ is to signify that it is a throwaway variable
    t = threading.Thread(target=do_something)   
    t.start()
    threads.append(t)

for thread in threads:
  thread.join()

finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

Sleeping for 1 second...
Sleeping for 1 second...
Sleeping for 1 second...
Sleeping for 1 second...
Sleeping for 1 second...
Sleeping for 1 second...
Sleeping for 1 second...
Sleeping for 1 second...
Sleeping for 1 second...Sleeping for 1 second...

Done Sleeping...
Done Sleeping...
Done Sleeping...
Done Sleeping...
Done Sleeping...
Done Sleeping...Done Sleeping...Done Sleeping...


Done Sleeping...
Done Sleeping...
Finished in 1.02 second(s)


Above we are running the `do_something` 10 times and it sleeps for 1 second every time. But, since we are using threads, it just going to move forward with the script. So, we can see that all of that go executed still within almost 1 second instead of taking 10 seconds.

Now, let us see how we can pass in arguments into our functions:

In [32]:
start = time.perf_counter()

def do_something_n(seconds):
  print(f'Sleeping for {seconds} second(s)...')
  time.sleep(seconds)
  print('Done Sleeping...')

threads = []

for _ in range(10):                 
    t = threading.Thread(target=do_something_n, args=[1.5]) # we need to pass a list containing the arguments to that function.  
    t.start()
    threads.append(t)

for thread in threads:
  thread.join()

finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

Sleeping for 1.5 second(s)...Sleeping for 1.5 second(s)...
Sleeping for 1.5 second(s)...

Sleeping for 1.5 second(s)...
Sleeping for 1.5 second(s)...
Sleeping for 1.5 second(s)...
Sleeping for 1.5 second(s)...Sleeping for 1.5 second(s)...

Sleeping for 1.5 second(s)...
Sleeping for 1.5 second(s)...
Done Sleeping...
Done Sleeping...Done Sleeping...

Done Sleeping...
Done Sleeping...
Done Sleeping...
Done Sleeping...Done Sleeping...

Done Sleeping...Done Sleeping...

Finished in 1.51 second(s)


In the above script, normally the script gets finished in 15 seconds, but here we complete it within 1.5 seconds almost.

After the so far implementation of threads, in `Python 3.2` onwards, we have thread pool executor and in a lot of cases, using that is more easy and efficient way to run threads. It also allows us to use multiple processes depending on the problem we are trying to solve.

Now let us use thread pool executors which is actually in the `concurrent.futures` module, and try to implement the above logic again.

In [33]:
import concurrent.futures   # importing concurrent.futures
import time

start = time.perf_counter()

def do_something_n(seconds):
    print(f'Sleeping {seconds} second(s)...')
    time.sleep(seconds)
    return f'Done Sleeping...'

with concurrent.futures.ThreadPoolExecutor() as executor:
    f1 = executor.submit(do_something_n, 1)  # the submit method schedules a function to be executed and returns a future object
    print(f1.result())   # the result() method will wait around till the functin completes and grabs the return value

finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

Sleeping 1 second(s)...
Done Sleeping...
Finished in 1.01 second(s)


A future object basically encapsulates the execution of our functiion and allows us to check in on it after it has been scheduled. 

In [34]:
import concurrent.futures   # importing concurrent.futures

start = time.perf_counter()

def do_something_n(seconds):
    print(f'Sleeping {seconds} second(s)...')
    time.sleep(seconds)
    return f'Done Sleeping...'

with concurrent.futures.ThreadPoolExecutor() as executor:
    f1 = executor.submit(do_something_n, 1)  # the submit method schedules a function to be executed and returns a future object
    f2 = executor.submit(do_something_n, 1)
    print(f1.result())   # the result() method will wait around till the functin completes and grabs the return value
    print(f2.result())

finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

Sleeping 1 second(s)...
Sleeping 1 second(s)...
Done Sleeping...
Done Sleeping...
Finished in 1.01 second(s)


Also if we create two executors, we see that both of them got kicked off almost the same time and returned the future object. Also we can loop over to execute it 10 times or a list comprehension as below: 

In [35]:
start = time.perf_counter()

with concurrent.futures.ThreadPoolExecutor() as executor:
  results = [executor.submit(do_something_n, 1) for _ in range(10)]  # we have created a list comprehension that is running our function 10 different times

  for f in concurrent.futures.as_completed(results):   
    # as_completed() gives us an iterator that we can loop over and yield the result of the threads as they are completed
    print(f.result())

finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

Sleeping 1 second(s)...
Sleeping 1 second(s)...
Sleeping 1 second(s)...
Sleeping 1 second(s)...Sleeping 1 second(s)...

Sleeping 1 second(s)...
Sleeping 1 second(s)...
Sleeping 1 second(s)...
Sleeping 1 second(s)...Sleeping 1 second(s)...

Done Sleeping...
Done Sleeping...
Done Sleeping...
Done Sleeping...
Done Sleeping...
Done Sleeping...
Done Sleeping...
Done Sleeping...
Done Sleeping...
Done Sleeping...
Finished in 1.02 second(s)


So it still took around 1 second to complete. Also to verify that they are printing when each future object is returned, we can keep checks by passing different values of seconds as below:

In [36]:
start = time.perf_counter()

def do_something_n(seconds):
    print(f'Sleeping {seconds} second(s)...')
    time.sleep(seconds)
    return f'Done Sleeping...{seconds}'

with concurrent.futures.ThreadPoolExecutor() as executor:
  secs = [5, 4, 3, 2, 1]
  results = [executor.submit(do_something_n, sec) for sec in secs]  # we have created a list comprehension that is running our function 10 different times

  for f in concurrent.futures.as_completed(results):   
    print(f.result())

finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

Sleeping 5 second(s)...
Sleeping 4 second(s)...
Sleeping 3 second(s)...
Sleeping 2 second(s)...Sleeping 1 second(s)...

Done Sleeping...1
Done Sleeping...2
Done Sleeping...3
Done Sleeping...4
Done Sleeping...5
Finished in 5.01 second(s)


Note that the order in which they are finished was 1->2->3->4->5 with the `submit()` method submitting each function once at a time, though we actually started the 5 second threa first but since we use `as_completed()` method it submitted our results in the order they were finished. And our total script took 5 seconds to complete.

We can refine the script above as follows using `map()` function:

In [40]:
start = time.perf_counter()

def do_something_n(seconds):
    print(f'Sleeping {seconds} second(s)...')
    time.sleep(seconds)
    return f'Done Sleeping...{seconds}'

with concurrent.futures.ThreadPoolExecutor() as executor:
  secs = [5, 4, 3, 2, 1]
  results = executor.map(do_something_n, secs)

  for result in results:
    print(result)

finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

Sleeping 5 second(s)...Sleeping 4 second(s)...

Sleeping 3 second(s)...Sleeping 2 second(s)...

Sleeping 1 second(s)...
Done Sleeping...5
Done Sleeping...4
Done Sleeping...3
Done Sleeping...2
Done Sleeping...1
Finished in 5.01 second(s)


We we used `submit()` method it returned a future object and when we use map it instead returns the results. In this it is still going to run the threads concurrently but instead of returning the results as they completed like before, `map()` is going to return the results in the order they started.

In the above output, the threads all get kicked off almost at the same time, and it looked like they all get completed at the same time. But, the did not actually completed all at the same time but when we loop over the results in the order they started. It still took 5 seconds altogether.

Now, below is an more realistic application of threading. In the script below we are downloading 15 images from a website. It isone of the I/O based operation. 

In [43]:
import requests

img_urls = [
    'https://images.unsplash.com/photo-1516117172878-fd2c41f4a759',
    'https://images.unsplash.com/photo-1532009324734-20a7a5813719',
    'https://images.unsplash.com/photo-1524429656589-6633a470097c',
    'https://images.unsplash.com/photo-1530224264768-7ff8c1789d79',
    'https://images.unsplash.com/photo-1564135624576-c5c88640f235',
    'https://images.unsplash.com/photo-1541698444083-023c97d3f4b6',
    'https://images.unsplash.com/photo-1522364723953-452d3431c267',
    'https://images.unsplash.com/photo-1513938709626-033611b8cc03',
    'https://images.unsplash.com/photo-1507143550189-fed454f93097',
    'https://images.unsplash.com/photo-1493976040374-85c8e12f0c0e',
    'https://images.unsplash.com/photo-1504198453319-5ce911bafcde',
    'https://images.unsplash.com/photo-1530122037265-a5f1f91d3b99',
    'https://images.unsplash.com/photo-1516972810927-80185027ca84',
    'https://images.unsplash.com/photo-1550439062-609e1531270e',
    'https://images.unsplash.com/photo-1549692520-acc6669e2f0c'
]

t1 = time.perf_counter()


for img_url in img_urls:
    img_bytes = requests.get(img_url).content  # using the request library 
    img_name = img_url.split('/')[3]   # parsing for the image name
    img_name = f'{img_name}.jpg'
    with open(img_name, 'wb') as img_file:   # opening a file in byte mode
        img_file.write(img_bytes)          # writing those image byues into the file 
        print(f'{img_name} was downloaded...')

t2 = time.perf_counter()

print(f'Finished in {t2-t1} seconds')

photo-1516117172878-fd2c41f4a759.jpg was downloaded...
photo-1532009324734-20a7a5813719.jpg was downloaded...
photo-1524429656589-6633a470097c.jpg was downloaded...
photo-1530224264768-7ff8c1789d79.jpg was downloaded...
photo-1564135624576-c5c88640f235.jpg was downloaded...
photo-1541698444083-023c97d3f4b6.jpg was downloaded...
photo-1522364723953-452d3431c267.jpg was downloaded...
photo-1513938709626-033611b8cc03.jpg was downloaded...
photo-1507143550189-fed454f93097.jpg was downloaded...
photo-1493976040374-85c8e12f0c0e.jpg was downloaded...
photo-1504198453319-5ce911bafcde.jpg was downloaded...
photo-1530122037265-a5f1f91d3b99.jpg was downloaded...
photo-1516972810927-80185027ca84.jpg was downloaded...
photo-1550439062-609e1531270e.jpg was downloaded...
photo-1549692520-acc6669e2f0c.jpg was downloaded...
Finished in 2.2150637459999416 seconds


Here it is actually spending a lot of time waiting. It is waiting for the response from the site and not moving to the next line until it get the entire response back from the site. Let us use threading for this:

In [42]:
t1 = time.perf_counter()


def download_image(img_url):
    img_bytes = requests.get(img_url).content
    img_name = img_url.split('/')[3]
    img_name = f'{img_name}.jpg'
    with open(img_name, 'wb') as img_file:
        img_file.write(img_bytes)
        print(f'{img_name} was downloaded...')


with concurrent.futures.ThreadPoolExecutor() as executor:  # it will request for the images in different threads, running it asynchronously
    executor.map(download_image, img_urls)

t2 = time.perf_counter()

print(f'Finished in {t2-t1} seconds')

photo-1516117172878-fd2c41f4a759.jpg was downloaded...
photo-1507143550189-fed454f93097.jpg was downloaded...
photo-1564135624576-c5c88640f235.jpg was downloaded...
photo-1530224264768-7ff8c1789d79.jpg was downloaded...
photo-1513938709626-033611b8cc03.jpg was downloaded...
photo-1524429656589-6633a470097c.jpg was downloaded...
photo-1516972810927-80185027ca84.jpg was downloaded...
photo-1504198453319-5ce911bafcde.jpg was downloaded...
photo-1522364723953-452d3431c267.jpg was downloaded...
photo-1532009324734-20a7a5813719.jpg was downloaded...
photo-1530122037265-a5f1f91d3b99.jpg was downloaded...
photo-1541698444083-023c97d3f4b6.jpg was downloaded...
photo-1493976040374-85c8e12f0c0e.jpg was downloaded...
photo-1549692520-acc6669e2f0c.jpg was downloaded...
photo-1550439062-609e1531270e.jpg was downloaded...
Finished in 1.8038244919989666 seconds


In [24]:
import logging
import threading
import time

def thread_function(name):
    logging.info("Thread %s: starting", name)
    time.sleep(2)
    logging.info("Thread %s: finishing", name)

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    logging.info("Main    : before creating thread")
    x = threading.Thread(target=thread_function, args=(1,))
    logging.info("Main    : before running thread")
    x.start()
    logging.info("Main    : wait for the thread to finish")
    # x.join()
    logging.info("Main    : all done")

09:45:12: Main    : before creating thread
09:45:12: Main    : before running thread
09:45:12: Thread 1: starting
09:45:12: Main    : wait for the thread to finish
09:45:12: Main    : all done


Now, we can see a bit of speed up in the process of downloading the images by the use of threads. The speed ups can be more drastic depending on what we are actually doing. 

## Difference between threading and multiprocessing:

Now, a lot of time we confuse between threading and multiprocessing: 
Though both are used to spped up multitasking, the ***threading module*** uses threads, the ***multiprocessing module*** uses processes. The difference is that threads run in the same memory space, while processes have separate memory. This makes it a bit harder to share objects between processes with multiprocessing.

![](https://www.lightbringercap.com/uploads/7/2/8/8/72888973/fg_orig.jpg)
***Image credits: [Data Science Central](https://www.google.com/url?sa=i&url=https%3A%2F%2Fwww.datasciencecentral.com%2Fprofiles%2Fblogs%2Fneuromancer-blues-threading-vs-multiprocessing&psig=AOvVaw21VHYGg3vjQQy1jjBplMW2&ust=1633689458942000&source=images&cd=vfe&ved=0CAwQjhxqFwoTCNj1r9SNuPMCFQAAAAAdAAAAABAb)***

## Conclusion

Also if some script is doing a lot of computation that would not be ideal for threading. In such cases it may decrease the spped of execution.

So depending upon the type of task we are doing we ned to chose the task. If it involves a lot of processing, we can use multi-processing instead of threading.

In the fields like game development, data science and ML/DL, threading have its extensive use, where we do not want to block the entire application due to the loading of any graphics, large data, plots or any other tasks.